Serial Schedule Generation Scheme (SGS)

The solver iteratively schedules jobs based on a priority rule. For each job, it determines the earliest feasible start time by considering:

* Precedence Constraints: Must start after all predecessors finish.
* Resource Constraints: Must start when sufficient resources are available for its entire duration.
* Latest Finish Time (LFT): Jobs are prioritized by their calculated LFT, aiming to schedule those with earlier deadlines first.

The algorithm maintains a resource timeline and incrementally searches for the first available time slot that satisfies all constraints. It also dynamically extends the timeline as needed and includes a mechanism to prevent deadlocks by ensuring resource availability.

Heuristics Employed:

* Latest Finish Time (LFT) Prioritization: Jobs are sorted by their LFT, with jobs having earlier LFTs being scheduled first. This aims to address time-critical tasks sooner.
* Earliest Feasible Start Time Search: The solver searches for the absolute earliest time a job can begin by incrementally checking time steps from its earliest possible start time until resource availability and precedence constraints are confirmed.
* Resource Availability Check & Jump: When a resource constraint is violated, the search for a start time jumps to the earliest point the blocking resource might become available, optimizing the search.
* Capacity Adjustment: If a job requires more resources than ever available, the system dynamically increases the resource capacity to prevent deadlocks, although this indicates an inherently infeasible project.

In [1]:
import json
import time
from typing import Dict, List

def solve_project(json_data: Dict) -> Dict:
    start_wall_time = time.time()
    
    # 1. Parse Input
    jobs = json_data.get('jobs', [])
    initial_res = json_data.get('initial_resources', {}) or {}
    
    # Convert all IDs to strings for consistency
    job_map = {str(j['id']): j for j in jobs}
    act_ids = [str(j['id']) for j in jobs]
    
    # Pre-calculate predecessors
    predecessors = {jid: [] for jid in act_ids}
    for j in jobs:
        jid = str(j['id'])
        successors = j.get('precedences', {}).get('time_successors', [])
        for succ in successors:
            if str(succ) in predecessors:
                predecessors[str(succ)].append(jid)

    # 2. Priority Rule: Latest Finish Time (LFT)
    total_duration = sum(j.get('duration', 0) for j in jobs)
    current_horizon = max(total_duration * 3, 500) 
    lf = {jid: current_horizon for jid in act_ids}
    
    for _ in range(len(act_ids)):
        for j in jobs:
            jid = str(j['id'])
            successors = j.get('precedences', {}).get('time_successors', [])
            valid_succs = [str(s) for s in successors if str(s) in job_map]
            if valid_succs:
                lf[jid] = min([lf[s] - job_map[s]['duration'] for s in valid_succs])

    # 3. Scheduling Scheme
    scheduled = {}
    res_timeline = {str(r): [amt] * (current_horizon + 1) for r, amt in initial_res.items()}
    
    def ensure_timeline_length(target_len: int):
        nonlocal current_horizon
        if target_len > current_horizon:
            extension = (target_len - current_horizon) + 500
            for r_id in res_timeline:
                res_timeline[r_id].extend([res_timeline[r_id][-1]] * extension)
            current_horizon = len(next(iter(res_timeline.values()))) - 1

    # Sort by LFT
    eligible_jobs = sorted(jobs, key=lambda x: lf[str(x['id'])])

    for job in eligible_jobs:
        jid = str(job['id'])
        dur = job.get('duration', 0)
        reqs = job.get('resources_required', {})
        cons_prod = job.get('resource_consumption', {})

        # Deadlock Prevention: Check if activity is EVER possible
        for r_id, amt in reqs.items():
            r_id_str = str(r_id)
            if r_id_str not in res_timeline:
                res_timeline[r_id_str] = [0] * (current_horizon + 1)
            
            # If demand > max possible capacity, we cap it or raise warning
            # (In a real scenario, this project would be impossible)
            max_capacity = max(res_timeline[r_id_str])
            if amt > max_capacity:
                # Forcefully increase capacity to prevent infinite loop
                res_timeline[r_id_str] = [max(val, amt) for val in res_timeline[r_id_str]]

        t_start = max([scheduled[p] + job_map[p]['duration'] for p in predecessors[jid]], default=0)
        
        # Search for a feasible window
        found_slot = False
        while not found_slot:
            ensure_timeline_length(t_start + dur + 1)
            
            feasible = True
            next_possible_t = t_start + 1
            
            for r_id, amt in reqs.items():
                r_id_str = str(r_id)
                for t in range(t_start, t_start + dur):
                    if res_timeline[r_id_str][t] < amt:
                        feasible = False
                        # Optimization: Jump to when this resource might be free
                        next_possible_t = max(next_possible_t, t + 1)
                        break
                if not feasible: break
            
            if feasible:
                scheduled[jid] = t_start
                # Update Renewable
                for r_id, amt in reqs.items():
                    for t in range(t_start, t_start + dur):
                        res_timeline[str(r_id)][t] -= amt
                # Update Consumption/Production
                for r_id, delta in cons_prod.items():
                    val = delta.get('amount', 0) if isinstance(delta, dict) else delta
                    r_id_str = str(r_id)
                    if r_id_str not in res_timeline:
                        res_timeline[r_id_str] = [0] * (current_horizon + 1)
                    for t in range(t_start + dur, len(res_timeline[r_id_str])):
                        res_timeline[r_id_str][t] += val
                found_slot = True
            else:
                t_start = next_possible_t
                # Safety break to prevent infinite hang if logic fails
                if t_start > current_horizon + 5000:
                    scheduled[jid] = t_start # Force schedule
                    found_slot = True

    # 4. Metrics
    finish_times = [scheduled[jid] + job_map[jid]['duration'] for jid in scheduled]
    makespan = max(finish_times) if finish_times else 0
    execution_time = (time.time() - start_wall_time)*1000

    return {
        "schedule": scheduled,
        "metrics": {
            "makespan": int(makespan),
            "total_penalty": 0,
            "execution_time": f"{execution_time:.6f}s"
        }
    }

In [2]:
with open("C:/Users/Admin/Desktop/nonrenewable.json", 'r') as f:
    data = json.load(f) 

result = solve_project(data)
print(result)

{'schedule': {'0': 0, '25': 0, '13': 0, '37': 0, '26': 0, '1': 0, '16': 0, '18': 0, '49': 0, '38': 0, '40': 0, '41': 4, '42': 6, '14': 10, '15': 10, '19': 11, '27': 8, '50': 10, '2': 0, '4': 0, '6': 7, '28': 0, '29': 4, '3': 7, '39': 14, '51': 20, '5': 10, '54': 11, '17': 21, '20': 21, '21': 27, '43': 14, '53': 28, '52': 37, '7': 13, '8': 9, '9': 10, '10': 17, '11': 21, '12': 28, '22': 42, '23': 37, '24': 45, '30': 8, '31': 16, '32': 16, '33': 16, '34': 18, '35': 0, '36': 25, '44': 24, '45': 24, '46': 0, '47': 0, '48': 31, '55': 39, '56': 45, '57': 43, '58': 49, '59': 52, '60': 61, '61': 61, '62': 61, '74': 61, '75': 61, '76': 61, '77': 61, '78': 61, '63': 61, '64': 61, '79': 66, '80': 68, '65': 69, '66': 70, '67': 75, '68': 84, '69': 92, '70': 84, '71': 84, '72': 85, '73': 99, '81': 76, '82': 82, '83': 82, '84': 86, '85': 96, '86': 99, '87': 99, '89': 99, '88': 107, '91': 99, '92': 108, '94': 115, '95': 125, '96': 108, '97': 134, '98': 134, '90': 134, '93': 139, '99': 134, '100': 140,